# ETL
El primer paso fue extraer las tablas de la página https://indicadores.enacom.gob.ar/datos-abiertos.

     Luego se abre el archivo .xlsx y se van transformando a .csv cada pestaña útil del archivo.
     No se convirtieron todas las tablas, porque varias tenían datos redundantes entre sí, a continuación se detalla cuáles de las 15 tablas se conservan
     
* Juntando **Penetracion-hogares** y **Penetracion-población** se consigue información más detallada que **Penetracion-totales**, por lo que se descarta esta última.

* **Acc_vel_loc_sinrango** se guarda por tener las localidades de forma más detallada.

* **Velocidad_sin_rango** tiene la misma información que Acceso por velocidad pero menos organizado así que solo guarda el segundo.

* **Velocidad % por prov** también se guarda.

* **Accesos_tecnologias_localidad** se guarda.

* Y la tabla **Ingresos** se guarda por tener información importante para el negocio.

      Finalmente con python se abren las tablas creando los data frames.


In [357]:
import pandas as pd
import numpy as np


## Carga
#### Acc_Vel_loc_sinrangos = veloc_loc

In [358]:
veloc_loc= pd.read_csv('..\internet\Internet.xlsx - Acc_vel_loc_sinrangos.csv', delimiter=',')
veloc_loc.head()

,Partido,Localidad,link Indec,Velocidad (Mbps),Provincia,Accesos
0,BUENOS AIRES,25 de Mayo,25 de Mayo,6854100,0.00,1
1,BUENOS AIRES,25 de Mayo,25 de Mayo,6854100,0.50,2
2,BUENOS AIRES,25 de Mayo,25 de Mayo,6854100,0.75,19
3,BUENOS AIRES,25 de Mayo,25 de Mayo,6854100,3.00,85
4,BUENOS AIRES,25 de Mayo,25 de Mayo,6854100,3.50,145


In [359]:
veloc_loc.rename(columns={'Partido': 'Provincia',
                          'Localidad': 'Partido',
                          'link Indec': 'Localidad',
                          'Provincia': 'Link Indec'} , inplace=True)
veloc_loc.head()

,Provincia,Partido,Localidad,Velocidad (Mbps),Link Indec,Accesos
0,BUENOS AIRES,25 de Mayo,25 de Mayo,6854100,0.00,1
1,BUENOS AIRES,25 de Mayo,25 de Mayo,6854100,0.50,2
2,BUENOS AIRES,25 de Mayo,25 de Mayo,6854100,0.75,19
3,BUENOS AIRES,25 de Mayo,25 de Mayo,6854100,3.00,85
4,BUENOS AIRES,25 de Mayo,25 de Mayo,6854100,3.50,145


In [360]:
print(veloc_loc.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18864 entries, 0 to 18863
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Provincia         18864 non-null  object 
 1   Partido           18864 non-null  object 
 2   Localidad         18863 non-null  object 
 3   Velocidad (Mbps)  18864 non-null  object 
 4   Link Indec        18864 non-null  float64
 5   Accesos           18857 non-null  object 
dtypes: float64(1), object(5)
memory usage: 884.4+ KB
None


In [361]:
# Eliminar duplicados
veloc_loc = veloc_loc.drop_duplicates()
veloc_loc.fillna(0, inplace=True)

In [362]:
veloc_loc['Velocidad (Mbps)'] = veloc_loc['Velocidad (Mbps)'].str.strip('Sin Datos').str.strip("'").replace('', np.nan).astype(float)
veloc_loc['Accesos']= veloc_loc['Accesos'].str.replace(',', '', regex=False ).astype(float)

In [363]:
print(veloc_loc.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18864 entries, 0 to 18863
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Provincia         18864 non-null  object 
 1   Partido           18864 non-null  object 
 2   Localidad         18864 non-null  object 
 3   Velocidad (Mbps)  18726 non-null  float64
 4   Link Indec        18864 non-null  float64
 5   Accesos           18857 non-null  float64
dtypes: float64(3), object(3)
memory usage: 884.4+ KB
None


In [364]:
# Guardar el dataset procesado
veloc_loc.to_csv('Acc_vel_loc_sinrango_procesado.csv', index=False)

## Carga
#### Accesos por tecnologia = Accesos_tec

In [365]:
Accesos_tec= pd.read_csv('..\internet\Internet.xlsx - Accesos Por Tecnología.csv', delimiter=',')

In [366]:
Accesos_tec.head()

,Año,Trimestre,Provincia,ADSL,Cablemodem,Fibra óptica,Wireless,Otros,Total
0,2024,2,Buenos Aires,"214,055","2,722,466","1,849,476","138,638","64,745","4,989,380"
1,2024,2,Capital Federal,"54,102","1,144,781","230,402","4,493","29,821","1,463,599"
2,2024,2,Catamarca,"4,951","10,303","58,355","1,384",81,"75,074"
3,2024,2,Chaco,"9,448","57,935","68,944","8,407","2,358","147,092"
4,2024,2,Chubut,"25,955","80,704","26,516","31,118","9,930","174,223"


In [367]:
Accesos_tec.tail()

,Año,Trimestre,Provincia,ADSL,Cablemodem,Fibra óptica,Wireless,Otros,Total
1004,2014,1,Santiago Del Estero,"32,567","3,598",19,915,34,"37,133"
1005,2014,1,Tierra Del Fuego,"21,618","2,837",648,1,"2,934","28,038"
1006,2014,1,Tucumán,"129,717",83,121,13,98,"130,032"
1007,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1008,*,Los datos provinciales no coinciden a nivel na...,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [368]:
Accesos_tec= Accesos_tec.drop([1007, 1008])

In [369]:
print(Accesos_tec.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1007 entries, 0 to 1006
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Año           1007 non-null   object
 1   Trimestre     1007 non-null   object
 2   Provincia     1007 non-null   object
 3   ADSL          1007 non-null   object
 4   Cablemodem    1007 non-null   object
 5   Fibra óptica  1007 non-null   object
 6   Wireless      1007 non-null   object
 7   Otros         1007 non-null   object
 8   Total         1007 non-null   object
dtypes: object(9)
memory usage: 70.9+ KB
None


In [370]:
Accesos_tec['Wireless'] = Accesos_tec['Wireless'].str.replace(',', '.').astype(float)
Accesos_tec['Cablemodem'] = Accesos_tec['Cablemodem'].str.replace(',', '').astype(float)
Accesos_tec['Fibra óptica'] = Accesos_tec['Fibra óptica'].str.replace(',', '').astype(float)
Accesos_tec['Otros'] = Accesos_tec['Otros'].str.replace(',', '').astype(float)
Accesos_tec['ADSL'] = Accesos_tec['ADSL'].str.replace(',', '').astype(float)
Accesos_tec['Total'] = Accesos_tec['Total'].str.replace(',', '').astype(float)
Accesos_tec['Año'] = Accesos_tec['Año'].str.strip('*').str.strip("'")
Accesos_tec['Año'] = Accesos_tec['Año'].str.strip(' ').str.strip("'")
Accesos_tec['Año'] = pd.to_datetime(Accesos_tec['Año'], format='%Y', errors='coerce')
Accesos_tec['Trimestre'] = Accesos_tec['Trimestre'].str.strip('*').str.strip("'")
Accesos_tec['Trimestre'] = Accesos_tec['Trimestre'].fillna(0).astype(int)


In [371]:
Accesos_tec.fillna(0, inplace=True)

In [372]:
print(Accesos_tec.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1007 entries, 0 to 1006
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   Año           1007 non-null   datetime64[ns]
 1   Trimestre     1007 non-null   int64         
 2   Provincia     1007 non-null   object        
 3   ADSL          1007 non-null   float64       
 4   Cablemodem    1007 non-null   float64       
 5   Fibra óptica  1007 non-null   float64       
 6   Wireless      1007 non-null   float64       
 7   Otros         1007 non-null   float64       
 8   Total         1007 non-null   float64       
dtypes: datetime64[ns](1), float64(6), int64(1), object(1)
memory usage: 70.9+ KB
None


In [373]:
# Guardar el dataset procesado
Accesos_tec.to_csv('Accesos Por Tecnología_procesado.csv', index=False)

## Carga
#### Ingresos = ingresos

In [374]:
ingresos = pd.read_csv('..\internet\Internet.xlsx - Ingresos .csv', delimiter=',')

In [375]:
ingresos.head()

,Año,Trimestre,Ingresos (miles de pesos),Periodo
0,2024,2,"442,032,166.69",Abr-Jun 2024
1,2024,1,"346,198,986.13",Ene-Mar 2024
2,2023,4,"167,376,014.80",Oct-Dic 2023
3,2023,3,"133,106,593.41",Jul-Sept 2023
4,2023,2,"118,060,280.28",Jun-Mar 2023


In [376]:
ingresos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42 entries, 0 to 41
Data columns (total 4 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   Año                        42 non-null     int64 
 1    Trimestre                 42 non-null     int64 
 2   Ingresos (miles de pesos)  42 non-null     object
 3    Periodo                   42 non-null     object
dtypes: int64(2), object(2)
memory usage: 1.4+ KB


In [377]:
ingresos['Año']=pd.to_datetime(ingresos['Año'], format='%Y', errors='coerce')
ingresos['Ingresos (miles de pesos)']=ingresos['Ingresos (miles de pesos)'].str.replace(',', '', regex=False ).astype(float)

In [378]:
ingresos.fillna(0, inplace=True)

In [379]:
ingresos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42 entries, 0 to 41
Data columns (total 4 columns):
 #   Column                     Non-Null Count  Dtype         
---  ------                     --------------  -----         
 0   Año                        42 non-null     datetime64[ns]
 1    Trimestre                 42 non-null     int64         
 2   Ingresos (miles de pesos)  42 non-null     float64       
 3    Periodo                   42 non-null     object        
dtypes: datetime64[ns](1), float64(1), int64(1), object(1)
memory usage: 1.4+ KB


In [380]:
#Guardar dataset procesado
ingresos.to_csv('Ingresos_Procesado.csv', index=False)

## Carga
#### Penetracion-hogares = P_hogares

In [381]:
P_hogares=pd.read_csv('..\internet\Internet.xlsx - Penetracion-hogares.csv', delimiter=',')

In [382]:
P_hogares.head()

,Año,Trimestre,Provincia,Accesos por cada 100 hogares
0,2024,2,Buenos Aires,79.84
1,2024,2,Capital Federal,116.37
2,2024,2,Catamarca,68.81
3,2024,2,Chaco,44.06
4,2024,2,Chubut,86.33


In [383]:
P_hogares.rename(columns={' Trimestre ' : 'Trimestre',
                          ' Provincia ': 'Provincia'}, inplace=True)

In [384]:
P_hogares.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 4 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   Año                           1000 non-null   int64  
 1   Trimestre                     1000 non-null   int64  
 2   Provincia                     1000 non-null   object 
 3   Accesos por cada 100 hogares  1000 non-null   float64
dtypes: float64(1), int64(2), object(1)
memory usage: 31.4+ KB


In [385]:
P_hogares['Año']= pd.to_datetime(P_hogares['Año'], format='%Y', errors='coerce')

In [386]:
P_hogares.fillna(0, inplace=True)

In [387]:
P_hogares.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 4 columns):
 #   Column                        Non-Null Count  Dtype         
---  ------                        --------------  -----         
 0   Año                           1000 non-null   datetime64[ns]
 1   Trimestre                     1000 non-null   int64         
 2   Provincia                     1000 non-null   object        
 3   Accesos por cada 100 hogares  1000 non-null   float64       
dtypes: datetime64[ns](1), float64(1), int64(1), object(1)
memory usage: 31.4+ KB


In [388]:
#Guardar el dataset procesado
P_hogares.to_csv('Penetracion-hogares_procesado.csv', index=False)

## Carga
#### Penetracion-poblacion = p_poblacion

In [389]:
p_poblacion = pd.read_csv('..\internet\Internet.xlsx - Penetración-poblacion.csv', delimiter=',')

In [390]:
p_poblacion.head()

,Año,Trimestre,Provincia,Accesos por cada 100 hab
0,2024,2,Buenos Aires,27.43
1,2024,2,Capital Federal,47.44
2,2024,2,Catamarca,17.50
3,2024,2,Chaco,11.78
4,2024,2,Chubut,26.46


In [391]:
p_poblacion.rename(columns={' Trimestre ':'Trimestre',
                                  ' Provincia ': 'Provincia'}, inplace=True)

In [392]:
p_poblacion.fillna(0, inplace=True)

In [393]:
p_poblacion.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 4 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Año                       1000 non-null   int64  
 1   Trimestre                 1000 non-null   int64  
 2   Provincia                 1000 non-null   object 
 3   Accesos por cada 100 hab  1000 non-null   float64
dtypes: float64(1), int64(2), object(1)
memory usage: 31.4+ KB


In [394]:
p_poblacion['Año']=pd.to_datetime(p_poblacion['Año'], format='%Y', errors='coerce')

In [395]:
p_poblacion.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 4 columns):
 #   Column                    Non-Null Count  Dtype         
---  ------                    --------------  -----         
 0   Año                       1000 non-null   datetime64[ns]
 1   Trimestre                 1000 non-null   int64         
 2   Provincia                 1000 non-null   object        
 3   Accesos por cada 100 hab  1000 non-null   float64       
dtypes: datetime64[ns](1), float64(1), int64(1), object(1)
memory usage: 31.4+ KB


In [396]:
#Guardar dataset procesado
p_poblacion.to_csv('Penetracion-poblacion_procesado.csv', index=False)

## Carga
### Velocidad % por prov = vel_prov

In [397]:
vel_prov=pd.read_csv('..\internet\Internet.xlsx - Velocidad % por prov.csv', delimiter=',')

In [398]:
vel_prov.fillna(0, inplace=True)

In [399]:
vel_prov.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1008 entries, 0 to 1007
Data columns (total 4 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Año                     1008 non-null   int64  
 1   Trimestre               1008 non-null   int64  
 2   Provincia               1008 non-null   object 
 3   Mbps (Media de bajada)  1008 non-null   float64
dtypes: float64(1), int64(2), object(1)
memory usage: 31.6+ KB


In [400]:
vel_prov['Año']=pd.to_datetime(vel_prov['Año'], format='%Y', errors='coerce')

In [401]:
vel_prov.head()

,Año,Trimestre,Provincia,Mbps (Media de bajada)
0,2024-01-01,2,Buenos Aires,157.41
1,2024-01-01,2,Capital Federal,233.01
2,2024-01-01,2,Catamarca,97.38
3,2024-01-01,2,Chaco,107.76
4,2024-01-01,2,Chubut,21.67


In [402]:
#Guardar dataset procesado
vel_prov.to_csv('Velocidad % por prov_procesado.csv', index=False)

## Carga
### Velocidad_sin_rangos = Vel_sin_rangos

In [403]:
Vel_sin_rangos=pd.read_csv('..\internet\Internet.xlsx - Velocidad_sin_Rangos.csv', delimiter=',')

In [404]:
Vel_sin_rangos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18884 entries, 0 to 18883
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Año          18884 non-null  int64 
 1   Trimestre    18884 non-null  int64 
 2   Provincia    18884 non-null  object
 3    Velocidad   18874 non-null  object
 4   Accesos      18884 non-null  object
dtypes: int64(2), object(3)
memory usage: 737.8+ KB


In [405]:
Vel_sin_rangos.fillna(0, inplace=True)

In [406]:
Vel_sin_rangos['Año']=pd.to_datetime(Vel_sin_rangos['Año'], format='%Y', errors='coerce')
Vel_sin_rangos[' Velocidad '] = Vel_sin_rangos[' Velocidad '].str.replace(',', '', regex=False ).str.strip(' - ').str.strip("'").replace('', np.nan).astype(float)
Vel_sin_rangos['Accesos']=Vel_sin_rangos['Accesos'].str.replace(',', '', regex=False ).astype(float)

In [407]:
Vel_sin_rangos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18884 entries, 0 to 18883
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   Año          18884 non-null  datetime64[ns]
 1   Trimestre    18884 non-null  int64         
 2   Provincia    18884 non-null  object        
 3    Velocidad   18858 non-null  float64       
 4   Accesos      18884 non-null  float64       
dtypes: datetime64[ns](1), float64(2), int64(1), object(1)
memory usage: 737.8+ KB


In [408]:
Vel_sin_rangos.head()

,Año,Trimestre,Provincia,Velocidad,Accesos
0,2024-01-01,2,BUENOS AIRES,75.0,1062.0
1,2024-01-01,2,BUENOS AIRES,59.0,59.0
2,2024-01-01,2,BUENOS AIRES,480.0,5.0
3,2024-01-01,2,BUENOS AIRES,4.0,41735.0
4,2024-01-01,2,BUENOS AIRES,18.0,1042.0


In [409]:
# Guardar dataset procesado
Vel_sin_rangos.to_csv('Velocidad_sin_Rangos_procesado.csv', index=False)

## Carga 
### Totales VMD = tot_vmd

In [410]:
tot_vmd =pd.read_csv('..\internet\Internet.xlsx - Totales VMD.csv', delimiter=',')

In [411]:
tot_vmd.head()

,Año,Trimestre,Mbps (Media de bajada),Trimestre.1
0,2024,2,139.25,Abr-Jun 2024
1,2024,1,139.15,Ene-Mar 2024
2,2023,4,139.04,Oct-Dic 2023
3,2023,3,129.67,Jul-Sept 2023
4,2023,2,123.95,Abr-Jun 2023


In [412]:
tot_vmd.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42 entries, 0 to 41
Data columns (total 4 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Año                     42 non-null     int64  
 1   Trimestre               42 non-null     int64  
 2   Mbps (Media de bajada)  42 non-null     float64
 3   Trimestre.1             42 non-null     object 
dtypes: float64(1), int64(2), object(1)
memory usage: 1.4+ KB


In [413]:
tot_vmd.fillna(0, inplace=True)

In [414]:
# Guardar dataset procesado
tot_vmd.to_csv('Total VMD_procesado.csv', index=False)